In [1]:
# todo:换个RTX8000（显存48g）的跑
from angle_emb import AnglE
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# peft_model_id = 'SeanLee97/angle-llama-7b-nli-v2'
peft_model_id = './models/angle-llama-13b-nli'
config = PeftConfig.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path).bfloat16().cuda()
model = PeftModel.from_pretrained(model, peft_model_id).cuda()

def decorate_text(text: str):
    return Prompts.A.format(text=text)

inputs = 'hello world!'
tok = tokenizer([decorate_text(inputs)], return_tensors='pt')
for k, v in tok.items():
    tok[k] = v.cuda()
vec = model(output_hidden_states=True, **tok).hidden_states[-1][:, -1].float().detach().cpu().numpy()
print(vec)

/opt/conda/envs/angle/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [06:00<00:00, 120.23s/it]
/opt/conda/envs/angle/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/angle/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag i

OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 106.94 MiB is free. Process 16124 has 23.58 GiB memory in use. Of the allocated memory 22.81 GiB is allocated by PyTorch, and 2.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
from angle_emb import AnglE, Prompts


peft_model_id = './models/angle-llama-13b-nli'
angle = AnglE.from_pretrained('models/NousResearch/Llama-2-13b-hf', pretrained_lora_path=peft_model_id)

print('All predefined prompts:', Prompts.list_prompts())
angle.set_prompt(prompt=Prompts.A)
print('prompt:', angle.prompt)
vec = angle.encode({'text': 'hello world'}, to_numpy=True)
print(vec)
vecs = angle.encode([{'text': 'hello world1'}, {'text': 'hello world2'}], to_numpy=True)
print(vecs)

INFO:AnglE:LLM detected, automatically set is_llm=True.If it is wrong, you can manually set `is_llm`.
INFO:AnglE:LLM detected, automatically set apply_lora=True.If it is wrong, you can manually set `apply_lora`.
INFO:AnglE:LLM detected, automatically set prompt. You can change this setting by manually configuring the `set_prompt()` function.
INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)
INFO:AnglE:LLaMA detected, automatically set `apply_bfloat16=True`. You can change this setting by manually configuring the `apply_bfloat16`.
Loading checkpoint shards: 100%|██████████| 3/3 [00:49<00:00, 16.43s/it]
/opt/conda/envs/angle/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
All predefined prompts: None
prompt: Summarize sentence "{text}" in one word:"


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [4]:
# bert
from angle_emb import AnglE

model_id = 'models/angle-bert-base-uncased-nli-en-v1'
angle = AnglE.from_pretrained(model_id, pooling_strategy='cls_avg').cuda()
vec = angle.encode('hello world', to_numpy=True)
print(vec)
vecs = angle.encode(['hello world1', 'hello world2'], to_numpy=True)
print(vecs)

[[-3.27171564e-01  1.75740778e-01  4.33561146e-01 -1.67173013e-01
   2.28020459e-01 -3.13170254e-01  3.23471576e-01  2.04031318e-01
  -4.91903007e-01 -3.83959204e-01 -3.45154583e-01 -2.40107596e-01
   2.18672417e-02  2.15509579e-01 -3.58560443e-01  1.87191486e-01
  -3.58546674e-01  4.50014293e-01 -6.13204837e-01  1.48928672e-01
  -1.20886594e-01 -2.03186542e-01  1.72497153e-01 -2.65070587e-01
  -2.46300057e-01 -1.50559723e-01  1.90194294e-01 -1.52787238e-01
  -1.75320074e-01 -1.45873457e-01 -5.22409439e-01 -4.29163456e-01
   3.16311032e-01 -4.92238142e-02 -5.73321223e-01  7.30248019e-02
  -4.08607781e-01  5.06453454e-01 -4.11549956e-01 -3.83612365e-01
   4.96911667e-02 -5.86191535e-01  3.62863153e-01  2.61388421e-01
  -5.26935518e-01 -6.30573869e-01 -8.98442984e-01  2.97703028e-01
  -4.99761760e-01 -1.85546607e-01  5.68788290e-01  2.64061615e-03
  -8.24039876e-02 -3.32786441e-01  1.18184894e-01 -2.99841851e-01
  -7.83518776e-02 -4.66939867e-01  2.29209363e-01 -3.58953774e-01
   2.86294